# Comparison of model-based and MRI-based target volume definition for ocular proton therapy 
These scripts provide methods to generate polynomial tumour models, as commonly used in ocular proton therapy planning, based on 3D tumour delineations. Furthermore, it provides methods to compare the resulting model to the tumour delineations. 

More figures and details about the methods are available within the functions.

L. Klaassen, 2025. 


In [1]:
from Prepare_base import correct_base, redefine_prom, expand_base
from Generate_tumour_model import generate_tumour_model, generate_tumour_model_extrathickness, upsample_point_cloud, save_point_cloud_as_stl, directed_angle_between_vectors
from Model_evaluation import volume_analysis, signed_surface_dist
from Automatic_measurements import Prom_Centre, LBD

import trimesh
import os
import numpy as np
import pandas as pd
import alphashape
import pymeshfix
import scipy
import warnings
import shapely
import matplotlib.pyplot as plt
import sklearn
import manifold3d
from tqdm.auto import tqdm 

### Preparations

NOTE: you should specify the path to the STL files below.

In [ ]:
# Define paths
filedir = r'' #Enter the path where STL files are
outdir = r'' # Enter the path where you want the models to go

tumour_path = os.path.join(filedir, 'Tumour_example.stl')
eye_path = os.path.join(filedir, 'Eye_example.stl')

if not os.path.isdir(outdir):
    os.makedirs(outdir)


In [ ]:
#Load stl
tumour = trimesh.load(tumour_path)
eye = trimesh.load(eye_path)

# Tumour measurements
thickness, thickness_base, thickness_top = Prom_Centre(tumour,eye, include_sclera = False) # Prominence = thickness
lbd, lbd_coor1, lbd_coor2, base = LBD(tumour,eye)
print('Thickness is ', round(thickness,1), 'mm, largest diameter is ', round(lbd,1), 'mm')

# Define tumour base and expanded tumour base
corrected_base, corrected_base_normals = correct_base(tumour,eye, threshold_angle = 45)
expanded_base, new_points = expand_base(corrected_base, max_distance = 1.0)

# Upsample point clouds for tumour base
corrected_base, new_points2 = upsample_point_cloud(corrected_base, factor=10, k=10) # Upsample base by factor 10 with 10 nearest neighbours

# Check if thickness needs to be corrected (automatic thickness determination is less accurate in tumours with middle of the eye within or very close to tumour)
thickness_base, redefined = redefine_prom(tumour,eye, corrected_base, thickness_base)
        

Thickness is  9.1 mm, largest diameter is  17.7 mm


### Construct tumour models

In [ ]:
# Construct tumour models: standard version without expansions
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)
        
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf))

        tumour_model_points = generate_tumour_model(tumour,eye, corrected_base, sf) #generate tumour model
        save_point_cloud_as_stl(tumour_model_points, out, max_attempts=5) #save tumour model as STL
        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points[:,0], tumour_model_points[:,1], tumour_model_points[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)


  0%|          | 0/9 [00:00<?, ?it/s]

   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf2.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf3.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Mesh not a volume, upsampling points (Attempt 3)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf6.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf7.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_sf9.stl


In [5]:
# Construct tumour models: version with extra thickness
for sf in tqdm(np.arange(1,10,1)): #Make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)

        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_sf{0}.stl'.format(sf))

        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrathickness = generate_tumour_model_extrathickness(tumour,eye, corrected_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness, out, max_attempts=5) #save tumour model as STL

        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness[:,0], tumour_model_points_extrathickness[:,1], tumour_model_points_extrathickness[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)
                

  0%|          | 0/9 [00:00<?, ?it/s]

   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf1.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf6.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf7.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf8.stl
   Mesh not a volume, upsampling points (Attempt 1)...
   Mesh not a volume, upsampling points (Attempt 2)...
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_sf9.stl


In [6]:
# Construct tumour models: version with expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)

        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrabase_sf{0}.stl'.format(sf))

        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrabase = generate_tumour_model(tumour,eye, expanded_base, sf) #generate tumour model 
        save_point_cloud_as_stl(tumour_model_points_extrabase, out, max_attempts=5) #save tumour model as STL

        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrabase[:,0], tumour_model_points_extrabase[:,1], tumour_model_points_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)


  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrabase_sf9.stl


In [7]:
# Construct tumour models: version with added thickness and expanded base
for sf in tqdm(np.arange(1,10,1)): #make models for shape factors 1-10 in steps of 1
    try:
        sf = round(sf,1)
        
        #location of files
        out = os.path.join(outdir, 'Testpatient_tumourmodel_extrathickness_extrabase_sf{0}.stl'.format(sf))
        
        tumour = trimesh.load(tumour_path)
        eye = trimesh.load(eye_path)

        tumour_model_points_extrathickness_extrabase = generate_tumour_model_extrathickness(tumour,eye, expanded_base, sf, addedthickness = 0.5) #generate tumour model with 0.5 mm added thickness
        save_point_cloud_as_stl(tumour_model_points_extrathickness_extrabase, out, max_attempts=5) #save tumour model as STL
        
        # Uncomment this section for a figure of the delineation and tumour model created
        #fig = plt.figure()
        #axes = fig.add_subplot(projection='3d')
        #axes.plot(eye.vertices[:,0], eye.vertices[:,1], eye.vertices[:,2], c='w', alpha = 0.8)
        #axes.scatter(tumour.vertices[:,0], tumour.vertices[:,1], tumour.vertices[:,2], c='g', alpha = 0.4, label = 'delineated tumour')
        #axes.scatter(tumour_model_points_extrathickness_extrabase[:,0], tumour_model_points_extrathickness_extrabase[:,1], tumour_model_points_extrathickness_extrabase[:,2], c= 'b', alpha = 0.7, label = 'tumour model')

        #axes.axis('equal')
        #axes.set_title('Tumour model and tumour delineation SF {0}'.format(sf))
        #fig.legend()

    except Exception as e: print(e)

  0%|          | 0/9 [00:00<?, ?it/s]

   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf1.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf2.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf3.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf4.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf5.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf6.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf7.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf8.stl
   Saved STL file: /tmp/Testpatient_tumourmodel_extrathickness_extrabase_sf9.stl


### Evaluate tumour models 
If tumour models have unexpected shapes, sometimes it is sufficient to run the model generation again for that shape factor.

In [8]:
# Evaluation for standard model
volume_metrics_perpatient = {}
distance_metrics_perpatient = {}

for sf in tqdm(np.arange(1,10,1)):
    try:
        sf = round(sf,1)
        tqdm.write('   started with sf {0}'.format(sf))
        model_path = os.path.join(outdir, 'Testpatient_tumourmodel_sf{0}.stl'.format(sf)) 
        model = trimesh.load(model_path)
        eye = trimesh.load(eye_path)

        volume_metrics = volume_analysis(tumour, model)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = volume_metrics

        dists, dist_metrics = signed_surface_dist(tumour, model, eye)
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = dist_metrics
        #print('    ', dist_metrics)
    except Exception as e:
        print(e)
        volume_metrics_perpatient[r'SF{0}'.format(sf)] = {'overlap_abs': np.nan, 'overlap_rel': np.nan, 'overestimation_abs': np.nan,
                'overestimation_rel': np.nan, 'underestimation_abs': np.nan,
                'underestimation_rel': np.nan, 'IoU': np.nan}
        distance_metrics_perpatient[r'SF{0}'.format(sf)] = {'surf_dist_median_abs':  np.nan, 'surf_dist_min': np.nan, 'surf_dist_perc_0.5': np.nan,'surf_dist_perc_1': np.nan,'surf_dist_perc_2': np.nan,
                                                            'surf_dist_perc_5':  np.nan, 'surf_dist_perc_25':  np.nan, 
                'surf_dist_perc_50':  np.nan, 'surf_dist_perc_75':  np.nan, 'surf_dist_perc_95':  np.nan, 'surf_dist_max': np.nan}


  0%|          | 0/9 [00:00<?, ?it/s]

   started with sf 1
   started with sf 2
   started with sf 3
   started with sf 4
   started with sf 5
   started with sf 6
   started with sf 7
   started with sf 8
   started with sf 9


In [9]:
# Put the results in a dataframe
rows = []
rows2 = []

for sf, metrics in volume_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows.append({
            'SF': sf_number,
            'overlap_abs': metrics['overlap_abs'],
            'overlap_rel': metrics['overlap_rel'],
            'overestimation_abs': metrics['overestimation_abs'],
            'overestimation_rel': metrics['overestimation_rel'],
            'underestimation_abs': metrics['underestimation_abs'],
            'underestimation_rel': metrics['underestimation_rel'],
            'IoU': metrics['IoU'],
        })


for sf, metrics in distance_metrics_perpatient.items():
    sf_number = float(sf.replace('SF', ''))
    rows2.append({
            'SF': sf_number,
            'surf_dist_median_abs': metrics['surf_dist_median_abs'],
            'surf_dist_min': metrics['surf_dist_min'],
            'surf_dist_perc_0.5': metrics['surf_dist_perc_0.5'],
            'surf_dist_perc_1': metrics['surf_dist_perc_1'],
            'surf_dist_perc_2': metrics['surf_dist_perc_2'],
            'surf_dist_perc_5': metrics['surf_dist_perc_5'],
            'surf_dist_perc_25': metrics['surf_dist_perc_25'],
            'surf_dist_perc_50': metrics['surf_dist_perc_50'],
            'surf_dist_perc_75': metrics['surf_dist_perc_75'],
            'surf_dist_perc_95': metrics['surf_dist_perc_95']
        })

volume_df = pd.DataFrame(rows)
distance_df = pd.DataFrame(rows2)

allmetrics_df = pd.DataFrame.merge(volume_df, distance_df, how = 'left')


In [10]:
# Export dataframe
out_excel_standardmodel = os.path.join(outdir, 'Evaluation_standardmodel.xlsx')
allmetrics_df.to_excel(out_excel_standardmodel)
allmetrics_df

,SF,overlap_abs,overlap_rel,overestimation_abs,overestimation_rel,underestimation_abs,underestimation_rel,IoU,surf_dist_median_abs,surf_dist_min,surf_dist_perc_0.5,surf_dist_perc_1,surf_dist_perc_2,surf_dist_perc_5,surf_dist_perc_25,surf_dist_perc_50,surf_dist_perc_75,surf_dist_perc_95
0,1.0,735.541473,67.811464,0.481828,0.044421,349.144554,32.188536,0.677814,1.390518,-2.350584,-2.282024,-2.245491,-2.189703,-2.080507,-1.496860,-0.696881,1.283863,2.027996
1,2.0,1038.562154,95.747722,26.955192,2.485069,46.123873,4.252279,0.934260,0.249286,-0.919069,-0.665488,-0.579487,-0.504240,-0.386423,-0.141071,0.175509,0.318674,0.489898
2,3.0,1071.958014,98.826572,186.299258,17.175409,12.728005,1.173428,0.843407,0.672562,-0.557284,-0.407481,-0.351164,-0.257639,0.090018,0.361815,0.672562,0.934710,1.307118
3,4.0,1070.986878,98.737040,303.892870,28.016667,13.699141,1.262959,0.771283,1.038720,-0.409727,-0.275073,-0.243282,-0.100304,0.151930,0.529387,1.038720,1.441147,1.869927
4,5.0,1075.455424,99.149007,388.729348,35.837960,9.230590,0.850992,0.729906,1.248449,-0.312261,-0.220606,-0.176604,0.075951,0.175473,0.649078,1.248449,1.789537,2.308463
5,6.0,1078.106429,99.393410,449.006908,41.395104,6.579597,0.606590,0.702948,1.377038,-0.350244,-0.204840,-0.148246,0.081496,0.186463,0.721022,1.377038,2.017193,2.618834
6,7.0,1076.180525,99.215856,493.130046,45.462930,8.505493,0.784143,0.682070,1.465551,-0.330476,-0.190421,-0.136215,0.102659,0.202046,0.764383,1.465551,2.173322,2.874401
7,8.0,1074.588947,99.069124,527.958968,48.673898,10.097077,0.930876,0.666352,1.527140,-0.401720,-0.162495,0.052361,0.118741,0.206740,0.802062,1.527140,2.251539,3.097252
8,9.0,1079.409559,99.513549,558.643334,51.502769,5.276469,0.486451,0.656843,1.562231,-0.371124,-0.163127,-0.088631,0.096257,0.203831,0.793703,1.562231,2.337349,3.310406
